In [103]:
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
path = "/content/drive/MyDrive/Colab Notebooks/chromedriver.exe"  
#driver = webdriver.Chrome('chromedriver')

# 옵션 생성
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [7]:
file_path = 'C:/Users/2hy03/Python_personal/Dx추천/DX콘텐츠.csv'
content = pd.read_csv(file_path, encoding='euc-kr')

content = content[(~content['url'].isna())]
url_list=content['url'].tolist()
content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 934 entries, 0 to 1042
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DX이해    918 non-null    object
 1   제목      934 non-null    object
 2   제작      904 non-null    object
 3   url     934 non-null    object
 4   부분      933 non-null    object
 5   분류1     638 non-null    object
 6   분류2     165 non-null    object
 7   분류3     22 non-null     object
 8   분류4     6 non-null      object
dtypes: object(9)
memory usage: 73.0+ KB


In [108]:
content.head()

,DX이해,제목,제작,url,부분,분류1,분류2,분류3,분류4
0,DX이해,1강. 디지털 트랜스포메이션(DX)에 대한 이해,KPC디지털,https://www.youtube.com/watch?v=TIW4r4FeOZE,DX기초,NaN,NaN,NaN,NaN
1,DX이해,2강. 디지털 트랜스포메이션을 위한 사전준비,KPC디지털,https://www.youtube.com/watch?v=zttI91TWqPs,DX기초,NaN,NaN,NaN,NaN
2,DX이해,3강. 디지털 역량 진단,KPC디지털,https://www.youtube.com/watch?v=dYr37tUnlvM,DX기초,NaN,NaN,NaN,NaN
3,DX이해,4강. 목표수립 및 방향설정,KPC디지털,https://www.youtube.com/watch?v=5IekH0xnth8,DX기초,NaN,NaN,NaN,NaN
4,DX이해,5강. 디지털 비즈니스 모델 혁신,KPC디지털,https://www.youtube.com/watch?v=7k4QEJNNoNs,DX기초,NaN,NaN,NaN,NaN


## DB수집
### 수집할 내용
- 콘텐츠 평가 데이터
    - 좋아요 수
    - 조회수
    - 댓글 수
- 콘텐츠 정보 데이터
    - 콘텐츠 키워드
    - 콘텐츠 시간
    - 콘텐츠 분류
    - 콘텐츠 언어
    - 제작자

In [104]:
df1=pd.read_csv('C:/Users/2hy03/Python_personal/Dx추천/DX_data1.csv',index_col=0)

In [210]:
for url in tqdm(url_list[150:200]):
    try:
        #driver = webdriver.Chrome('chromedriver')
        driver = webdriver.Chrome('chromedriver', options=options)
        driver.get(url)
        time.sleep(3)

        try:
                #더보기 버튼 예외처리
                expand_button = driver.find_elements(By.XPATH, '//*[@id="expand"]')
                if expand_button:
                    expand_button[0].click()
                    
                # html소스 가져오기 
                time.sleep(5)
                html = driver.page_source
                dsoup = BeautifulSoup(html, 'html.parser')
                    
                    
                # 제목
                title = dsoup.find('yt-formatted-string', class_='style-scope ytd-watch-metadata').string
                    
                # 날짜
                date = dsoup.select('yt-formatted-string#info')[0].text
                pattern = r"\d{4}\.\d{1,2}\.\d{1,2}"
                date=date.replace(" ","")
                match = re.search(pattern, date)
                if match:
                    date = match.group()
                    
                    # 키워드
                try:
                    keyword = dsoup.find('a', class_='yt-simple-endpoint style-scope yt-formatted-string bold').text
                except AttributeError:
                    keyword = ''
                    
                # 제작자
                creator=dsoup.find('yt-formatted-string', id='text').string
                    
                # 조회수
                view = dsoup.find('span', class_='style-scope yt-formatted-string bold').string
                view = re.sub(r'\D', '', view)
                    
                # 좋아요 수
                likes = dsoup.find(class_='style-scope ytd-segmented-like-dislike-button-renderer').text
                likes = re.sub(r'\D', '', likes)
                    
                #설명
                try:
                    text = dsoup.find(class_='yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap').text
                except AttributeError:
                    text=''
                    
                # 데이터프레임에 행 추가
                df1 = df1.append({'title': title, 'url': url, 'date': date, 'keyword': keyword, 'creator': creator, 'view': view, 'likes': likes,'text' : text}, ignore_index=True)

                #driver.quit()

        except ElementNotInteractableException:
                    driver.quit()
    except  ConnectionRefusedError:
        print(f"Connection refused for URL: {url}")
        continue

100%|██████████| 31/31 [06:11<00:00, 12.00s/it]


In [213]:
df1.to_csv('DX_data2.csv', index=False)

In [17]:
# 중복된 제목 컬럼 삭제
df1.drop_duplicates(subset='title', inplace=True)
df1

,title,url,date,keyword,creator,view,likes,text
0,1강. 디지털 트랜스포메이션(DX)에 대한 이해,https://www.youtube.com/watch?v=TIW4r4FeOZE,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,2508,29.0,제 1강 ‘디지털 트랜스포메이션(DX)에 대한 이해‘ 강의입니다. \n1강에서는 디...
1,2강. 디지털 트랜스포메이션을 위한 사전준비,https://www.youtube.com/watch?v=zttI91TWqPs,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,764,11.0,제 2강 ‘디지털 트랜스포메이션을 위한 사전준비‘ 강의입니다. \n2강에서는 어떻게...
2,3강. 디지털 역량 진단,https://www.youtube.com/watch?v=dYr37tUnlvM,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,799,9.0,제 3강 ‘디지털 역량 진단‘ 강의입니다. \n3강에서는 디지털 트랜스포메이션을 위...
3,4강. 목표수립 및 방향설정,https://www.youtube.com/watch?v=5IekH0xnth8,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,504,9.0,제 4강 ‘목표수립 및 방향설정‘ 강의입니다. \n4강에서는 디지털 트랜스포메이션을...
4,5강. 디지털 비즈니스 모델 혁신,https://www.youtube.com/watch?v=7k4QEJNNoNs,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,405,5.0,제 5강 ‘디지털 비즈니스 모델 혁신‘ 강의입니다. \n5강에서는 디지털 비즈니스 ...
...,...,...,...,...,...,...,...,...
193,우리 기업의 DX 성공 이야기 ⑧ - 한국신용데이터,https://www.youtube.com/watch?v=tWvu7_s4I7g&pp...,2023.1.8,#DX,한국무역협회 KITA TV,243,5.0,- 본 영상은 웨인힐스브라이언트 AI 인공지능 기반 콘텐츠로 제작되었습니다-\n우리...
194,우리 기업의 DX 성공 이야기 ⑬ - KT,https://www.youtube.com/watch?v=2HCVivrEHz0,2023.2.26,#DX,한국무역협회 KITA TV,332,4.0,- 본 영상은 웨인힐스브라이언트 AI 인공지능 기반 콘텐츠로 제작되었습니다-\n우리...
195,우리 기업의 DX 성공 이야기 ⑫ - 윌로그,https://www.youtube.com/watch?v=EFNU0KLRvEI,2023.2.19,#DX,한국무역협회 KITA TV,270,3.0,- 본 영상은 웨인힐스브라이언트 AI 인공지능 기반 콘텐츠로 제작되었습니다-\n우리...
196,우리 기업의 DX 성공 이야기 ⑩ - 에스이임파워,https://www.youtube.com/watch?v=zObnv296vpY,2023.1.29,#DX,한국무역협회 KITA TV,203,1.0,- 본 영상은 웨인힐스브라이언트 AI 인공지능 기반 콘텐츠로 제작되었습니다-\n우리...


## 
